## This is a notebook to recombine NER entities into one

In [1]:
import pandas as pd
import re
import numpy as np

In [49]:
df = pd.read_csv("landmine_tweets_ner_results_eng.csv")

In [50]:
df.shape

(5103, 8)

In [34]:
df['tagged_text'] = df['tagged_text'].fillna(" ")

In [35]:
#df = df[df["tagged_text"] != "RT"].reset_index()

In [36]:
df.shape

(5103, 8)

In [47]:
#df = clean_tweets(df, "tagged_text")

In [45]:
df['tagged_text'] = df["tagged_text"].str.replace("##", "")

In [37]:
# function to combine ner tags
def combine_ner_tags(df, spacer):
    """
    Function to recombine NER tags from B- to I-
    df = dataframe
    spacer = what kind of spacing to use to combine text. Use "" for non english and " " for english
    """
    combined_tokens = []
    cur_tokens = [df.loc[0, "tagged_text"]]
    for i in range(1, len(df)):
        if df.loc[i, "ner_tag"].startswith("B-"):
            combined_tokens.append(cur_tokens)
            cur_tokens = [df.loc[i, "tagged_text"]]
        elif df.loc[i, "ner_tag"].startswith("I-"):
             cur_tokens.append(df.loc[i, "tagged_text"])
    b_tags = df[df["ner_tag"].str.startswith("B-")]
    combined_tokens.append([b_tags.tail(1)["tagged_text"].values[0]])
    # update the combined text join based on which language. no space for non_eng
    combined_text = [spacer.join(i) for i in combined_tokens]
    b_tags["tagged_text"] = combined_text
    return b_tags

In [23]:
def combine_ner_tags(df):
    combined_tokens = []
    cur_tokens = [df.loc[0, "tagged_text"]]
    for i in range(1, len(df)):
        if df.loc[i, "ner_tag"].startswith("B-"):
            combined_tokens.append(cur_tokens)
            cur_tokens = [df.loc[i, "tagged_text"]]
        elif df.loc[i, "ner_tag"].startswith("I-"):
             cur_tokens.append(df.loc[i, "tagged_text"])
    b_tags = df[df["ner_tag"].str.startswith("B-")]
    combined_tokens.append([b_tags.tail(1)["tagged_text"].values[0]])
    # update the combined text join based on which language. no space for non_eng
    combined_text = [' '.join(i) for i in combined_tokens]
    b_tags["tagged_text"] = combined_text
    return b_tags

In [38]:
df_combined = combine_ner_tags(df, " ")

/tmp/ipykernel_761/1601176712.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b_tags["tagged_text"] = combined_text


In [39]:
df_combined.shape

(2530, 8)

In [40]:
df_combined = df_combined[df_combined["tagged_text"] != ""]

In [41]:
df_combined = df_combined[df_combined["tagged_text"] != " "]

In [42]:
df_combined.shape

(2527, 8)

In [44]:
df_combined.head()

,document_id,sentence_number,sentence_text,tagged_text,ner_tag,tagged_text_loci,ner_model,probability
0,1635764993353195526,1,rt an israeli motorist was seriously wounded b...,israeli,B-person,2,tner/bert-large-tweetner7-2020,0.996506
1,1635764993353195526,1,rt an israeli motorist was seriously wounded b...,bomb,B-event,9,tner/bert-large-tweetner7-2020,0.679351
2,1635764993353195526,1,rt an israeli motorist was seriously wounded b...,north,B-location,12,tner/bert-large-tweetner7-2020,0.999964
3,1635742122350444550,1,rt an israeli motorist was seriously wounded b...,israeli,B-person,2,tner/bert-large-tweetner7-2020,0.996506
4,1635742122350444550,1,rt an israeli motorist was seriously wounded b...,bomb,B-event,9,tner/bert-large-tweetner7-2020,0.679351


In [31]:
df_combined.to_csv("landmine_tweets_ner_results_eng_clean.csv", index = False)